In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import  AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from torch import  nn
import torch.nn.functional as F
import re
import demoji

In [2]:
df = pd.read_csv("cleaned_PS_train.csv")
le = LabelEncoder()
df['encoded_labels'] = le.fit_transform(df['labels'])
num_classes = len(le.classes_)
original_df = df.copy()
df

,content,labels,encoded_labels
0,தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் ...,Neutral,1
1,அண்ணன் இதனை சூசகமாக 11 மாதங்கள் முன்பே பேட்டிய...,Substantiated,6
2,ஒரு வருடம் ஆகி விட்டது இந்த துயரம் நேர்ந்து......,Opinionated,3
3,எடப்பாடியை கண்டுகொள்ளாத எடப்பாடி🫢 --- ஆதரிப்பீ...,Positive,4
4,எங்களின் அரசியல் அடுத்த தலைமுறைக்குமானது <hash...,Opinionated,3
...,...,...,...
4347,"பள்ளி , கல்லூரி படிப்பை இலவசமாக தாருங்கள் பேரு...",Substantiated,6
4348,இஸ்லாமியர்களின் பாதுகாவலர்கள் திமுகவினரா? உண்ம...,Opinionated,3
4349,டெல்டா மாவட்ட மக்களோடு... மக்களாக :smiling fac...,Positive,4
4350,நன்றி,Opinionated,3


In [3]:
df

,content,labels,encoded_labels
0,தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் ...,Neutral,1
1,அண்ணன் இதனை சூசகமாக 11 மாதங்கள் முன்பே பேட்டிய...,Substantiated,6
2,ஒரு வருடம் ஆகி விட்டது இந்த துயரம் நேர்ந்து......,Opinionated,3
3,எடப்பாடியை கண்டுகொள்ளாத எடப்பாடி🫢 --- ஆதரிப்பீ...,Positive,4
4,எங்களின் அரசியல் அடுத்த தலைமுறைக்குமானது <hash...,Opinionated,3
...,...,...,...
4347,"பள்ளி , கல்லூரி படிப்பை இலவசமாக தாருங்கள் பேரு...",Substantiated,6
4348,இஸ்லாமியர்களின் பாதுகாவலர்கள் திமுகவினரா? உண்ம...,Opinionated,3
4349,டெல்டா மாவட்ட மக்களோடு... மக்களாக :smiling fac...,Positive,4
4350,நன்றி,Opinionated,3


In [4]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, logits, targets):
        ce_loss = F.cross_entropy(logits, targets, reduction='none')
        p_t = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - p_t) ** self.gamma * ce_loss
        return focal_loss.mean()

In [5]:
class TamilTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def compute_metrics(predictions, labels):
    if not isinstance(predictions, torch.Tensor):
        predictions = torch.tensor(predictions)
    if not isinstance(labels, torch.Tensor):
        labels = torch.tensor(labels)
    
    correct = (predictions == labels).sum().item()
    total = len(labels)
    accuracy = correct / total
    return accuracy


In [6]:
class TamilTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [7]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/tamil-sentence-similarity-sbert",cache_dir='models/tamil-sentence-similarity-sbert')
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/tamil-sentence-similarity-sbert",num_labels=num_classes,cache_dir='models/tamil-sentence-similarity-sbert').to('cuda')

text = "தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் டாக்டர்"
tokens = tokenizer(text, return_tensors="pt").to('cuda')
print(tokens)

output = model(**tokens)
print(output)
print(type(output))
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/tamil-sentence-similarity-sbert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[  104, 66207,  7157,  3251, 36834,  7264, 64011, 16335,   105]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0004, -0.0008, -0.0041,  0.0035, -0.0053, -0.0040,  0.0015]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
<class 'transformers.modeling_outputs.SequenceClassifierOutput'>
Total number of parameters: 237561607


In [8]:
config = model.config
print(config)

BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "l3cube-pune/tamil-sentence-similarity-sbert",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache":

In [9]:
import torch
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm

def compute_metrics(predictions, labels):
    """
    Compute accuracy, precision, recall, and F1 score.
    """
    predictions = predictions.numpy()
    labels = labels.numpy()

    accuracy = (predictions == labels).mean()
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    return accuracy, precision, recall, f1

# Define the training function
def train_model(model, train_loader, val_loader, device, num_epochs):
    optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )
    focal_loss = FocalLoss(alpha=1.0, gamma=2.0).to(device)

    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')

        # Training
        model.train()
        total_train_loss = 0
        train_predictions = []
        train_true_labels = []

        for batch in tqdm(train_loader, desc='Training'):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=None
            )

            loss = focal_loss(outputs.logits, labels)
            total_train_loss += loss.item()

            predictions = torch.argmax(outputs.logits, dim=1)
            train_predictions.extend(predictions.cpu())
            train_true_labels.extend(labels.cpu())

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_loader)
        train_accuracy, train_precision, train_recall, train_f1 = compute_metrics(
            torch.stack(train_predictions),
            torch.stack(train_true_labels)
        )

        print(f"Train Loss: {avg_train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1 Score: {train_f1:.4f}")

        # Validation
        model.eval()
        total_val_loss = 0
        val_predictions = []
        val_true_labels = []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validation'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=None
                )

                loss = focal_loss(outputs.logits, labels)
                total_val_loss += loss.item()

                predictions = torch.argmax(outputs.logits, dim=1)
                val_predictions.extend(predictions.cpu())
                val_true_labels.extend(labels.cpu())

        avg_val_loss = total_val_loss / len(val_loader)
        val_accuracy, val_precision, val_recall, val_f1 = compute_metrics(
            torch.stack(val_predictions),
            torch.stack(val_true_labels)
        )

        print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1 Score: {val_f1:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            print('Saving best model...')
            torch.save(model.state_dict(), 'bestmodels/tamilsbert_nohashtags_best_model.pt')


In [10]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['encoded_labels'])

train_dataset = TamilTextDataset(
        texts=train_df['content'].values,
        labels=train_df['encoded_labels'].values,
        tokenizer=tokenizer
    )
    
val_dataset = TamilTextDataset(
    texts=val_df['content'].values,
    labels=val_df['encoded_labels'].values,
    tokenizer=tokenizer
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)
val_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0
    )
    
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(197285, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [11]:
print(train_dataset[20]['input'])
print(tokenizer.decode(train_dataset[20]['input_ids']))

கோவை கள நிலவரம்... நெருக்கடியில் இரு பெரிய கட்சிகள்.. ட்விஸ்ட் மேல் ட்விஸ்ட்..! watch full video link <hashtag/> <hashtag/> <hashtag/>
[CLS] கோவை கள நிலவரம்... நெருக்கடியில் இரு பெரிய கட்சிகள்.. ட்விஸ்ட் மேல் ட்விஸ்ட்..! watch full video link < hashtag / > < hashtag / > < hashtag / > [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [12]:
train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        num_epochs=30
    )


Epoch 1/30


Training: 100%|██████████| 490/490 [03:57<00:00,  2.06it/s]
c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Train Loss: 1.3125, Accuracy: 0.3098, Precision: 0.1458, Recall: 0.3098, F1 Score: 0.1497


Validation: 100%|██████████| 55/55 [00:10<00:00,  5.43it/s]
c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Loss: 1.2715, Accuracy: 0.3119, Precision: 0.0973, Recall: 0.3119, F1 Score: 0.1483
Saving best model...

Epoch 2/30


...

Training: 100%|██████████| 490/490 [05:31<00:00,  1.48it/s]
c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Train Loss: 0.5860, Accuracy: 0.5899, Precision: 0.4787, Recall: 0.5899, F1 Score: 0.4961


Validation: 100%|██████████| 55/55 [00:15<00:00,  3.55it/s]

Validation Loss: 1.6232, Accuracy: 0.3028, Precision: 0.2627, Recall: 0.3028, F1 Score: 0.2646



c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
df

,content,labels,encoded_labels
0,தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் ...,Neutral,1
1,அண்ணன் இதனை சூசகமாக 11 மாதங்கள் முன்பே பேட்டிய...,Substantiated,6
2,ஒரு வருடம் ஆகி விட்டது இந்த துயரம் நேர்ந்து......,Opinionated,3
3,"எடப்பாடியை கண்டுகொள்ளாத ""எடப்பாடி""🫢\n ---\nஆதர...",Positive,4
4,எங்களின் அரசியல் அடுத்த தலைமுறைக்குமானது \n#மக...,Opinionated,3
...,...,...,...
4347,"பள்ளி , கல்லூரி படிப்பை இலவசமாக தாருங்கள் பேரு...",Substantiated,6
4348,இஸ்லாமியர்களின் பாதுகாவலர்கள் திமுகவினரா? உண்ம...,Opinionated,3
4349,டெல்டா மாவட்ட மக்களோடு... மக்களாக 😍\n\n#மக்களி...,Positive,4
4350,நன்றி,Opinionated,3
